<a href="https://colab.research.google.com/github/z0104241/slm/blob/main/HyperclovaX_3b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install decord
!pip install -U bitsandbytes
!pip install av

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 122.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.2/35.2 MB 17.0 MB/s eta 0:00:00


In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoProcessor, AutoTokenizer, BitsAndBytesConfig

# 모델, 프로세서, 토크나이저 로드
model_name = "naver-hyperclovax/HyperCLOVAX-SEED-Vision-Instruct-3B"

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 프로세서 로드
processor = AutoProcessor.from_pretrained(model_name, trust_remote_code=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
# 양자화 없이 모델 로드
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    device_map="auto"  # 자동으로 가용 GPU에 배치
)

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:  39%|###8      | 3.16G/8.15G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:  35%|###5      | 2.71G/7.63G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:  37%|###6      | 2.87G/7.84G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/69.0 [00:00<?, ?B/s]

In [1]:
def ask_model(question):
    # 토크나이저를 사용하여 텍스트만 처리
    inputs = tokenizer(
        question,
        return_tensors="pt"
    ).to(model.device)

    # 추론 설정
    generation_config = {
        "max_new_tokens": 200,
        "do_sample": True,
        "temperature": 0.7,
        "top_p": 0.9,
    }

    # 모델을 통한 추론
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            **generation_config
        )

    # 토큰화된 출력을 텍스트로 변환
    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # 응답 중에서 질문 이후 부분만 추출 (모델 출력 형식에 따라 조정 필요)
    if question in response:
        response = response.split(question)[-1].strip()

    return response



In [ ]:
question1 = "2.12와 2.9 둘 중 어떤 수가 더 클까?"
answer1 = ask_model(question1)
print(f"질문: {question1}")
print(f"응답: {answer1}")



질문: 2.12와 2.9 둘 중 어떤 수가 더 클까?
응답: **
   - 2.12와 2.9를 비교하면, 소수점 아래 숫자를 비교해야 합니다.
   - 2.12의 소수점 아래 숫자는 2이고, 2.9의 소수점 아래 숫자는 9입니다.
   - 2 < 9이므로, 2.12가 더 작습니다.
   - 따라서 2.9가 더 큽니다.

2. **3.5와 1.8 둘 중 어떤 수가 더 클까?**
   - 3.5와 1.8을 비교하면, 소수점 아래 숫자를 비교해야 합니다.
   - 3.5의 소수점 아래 숫자는 5이고, 1.8의 소수점 아래 숫자는 8입니다.
   - 5 < 8이므로, 1.8이 더 큽니다.
   - 따라서 3.5가 더 작습니다.

3. **


In [ ]:
question2 = "**코너**는 **토푸리아**보다 키가 크고 **토푸리아**는 **메랍**보다 키가 크다. 3명 중 키가 가장 큰 사람은 누구인가?"
answer2 = ask_model(question2)
print(f"\n질문: {question2}")
print(f"응답: {answer2}")


질문: **코너**는 **토푸리아**보다 키가 크고 **토푸리아**는 **메랍**보다 키가 크다. 3명 중 키가 가장 큰 사람은 누구인가?
응답: **
- **토푸리아**: 170 cm
- **코너**: 175 cm
- **메랍**: 165 cm

토푸리아는 메랍보다 키가 크고, 코너는 토푸리아보다 키가 크므로, **코너**가 **토푸리아**보다 키가 크고, **토푸리아**는 **메랍**보다 키가 크므로 **코너**가 **메랍**보다 키가 크다. 따라서 키 순서는 **코너** > **토푸리아** > **메랍**이다.

따라서 정답은 **코너**입니다.


In [8]:
# Image 1
import torch
import re
from transformers import TextIteratorStreamer
import threading

def ask_clova_stream(query: str, device="cuda",
                    max_new_tokens=512, top_p=0.6, temperature=0.5, repetition_penalty=1.0):
    """
    query를 넣으면 모델 답변을 스트리밍하면서 출력하는 함수
    """

    # 1. Chat 포맷 구성
    chat = [
        {"role": "system", "content": "you are helpful assistant!"},
        {"role": "user", "content": query},
    ]

    # 2. 입력 토크나이즈
    input_ids = tokenizer.apply_chat_template(chat, return_tensors="pt", tokenize=True)
    input_ids = input_ids.to(device)

    # 3. Streamer 준비
    streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

# Image 2
    # 4. 모델 생성 (별도 스레드에서 실행)
    generation_kwargs = dict(
        input_ids=input_ids,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        top_p=top_p,
        temperature=temperature,
        repetition_penalty=repetition_penalty,
        streamer=streamer,
    )

    thread = threading.Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()

# Image 3
    # 5. 스트림 수신
    collected_text = ""
    for new_text in streamer:
        print(new_text, end="", flush=True)  # 한 글자 한 글자 바로 출력
        collected_text += new_text

    return collected_text

In [9]:
korean_qa = {
    "한국의 수도는 어디인가요?": ["서울", "서울특별시"],
    "한국에 제일높은 무엇인가요?": ["백두산", "고조봉"],
    "한글을 창제한 왕은 누구인가요?": ["세종대왕", "세종", "이도"],
    "한국의 국화는 무엇인가요?": ["무궁화"],
    "한라도는 어느 나라의 무엇인가요?": ["한국", "대한민국"],
    "한국의 전통 집 양식은 무엇이라고 하나요?": ["한옥"],
    "삼일절은 무엇을 기념하는 날인가요?": ["3.1 운동", "삼일 운동", "독립 만세 운동"],
    "한국의 전통 의복은 무엇인가요?": ["한복"],
    "한국에서 가장 높은 산은 무엇인가요?": ["백두산", "한라산"],
    "한국의 국보 1호는 무엇인가요?": ["숭례문", "남대문"]
}

# 모델 응답 수집 함수
def collect_model_responses(model_func, questions):
    responses = {}
    for question in questions:
        response = model_func(question)
        responses[question] = response
    return responses

# 모델별 응답 수집
clova_responses = {}  # ask_clova_stream 함수로 수집한 예정


In [11]:
# 응답 수집 여서
clova_responses = collect_model_responses(ask_clova_stream, korean_qa.keys())

assistant
한국의 수도는 서울입니다.assistant
한국에서 가장 높은 산은 한라산입니다. 한라산은 해발 1,947미터로, 대한민국에서 가장 높은 산이며, 제주도에 위치해 있습니다.assistant
한글을 창제한 왕은 세종대왕입니다. 세종대왕은 조선의 4대 왕으로, 1443년에 한글을 창제하고 1446년에 반포하였습니다. 한글은 당시 백성들이 한자를 배우기 어려워하는 문제를 해결하기 위해 창제되었으며, 백성들이 쉽게 읽고 쓸 수 있는 문자를 제공하는 것을 목적으로 하였습니다. 한글은 과학적이고 체계적인 문자로 평가받고 있으며, 한국 문화와 정체성을 상징하는 중요한 유산 중 하나로 여겨지고 있습니다.assistant
한국의 국화는 **무궁화**입니다. 무궁화는 오랜 역사를 가진 꽃으로, 한국의 아름다움과 강인함을 상징합니다.assistant
한라도는 제주도를 의미하는 말로, 대한민국의 제주특별자치도를 가리킵니다. 제주도는 한국에서 가장 큰 섬으로, 아름다운 자연 경관과 독특한 문화로 유명합니다. 한라산은 제주도의 중심에 위치한 화산으로, 해발 1,950미터의 높이를 자랑하며, 많은 등산객들이 찾는 명소입니다.assistant
한국의 전통 집 양식은 '한옥'이라고 합니다. 한옥은 주로 나무를 사용하여 지으며, 기와로 지붕을 덮는 것이 특징입니다. 또한, 한옥은 자연과 조화를 이루며, 온돌(바닥 난방) 시스템을 갖추고 있어 겨울에도 따뜻하게 지낼 수 있습니다. 한옥의 창호는 한지(한국 전통 종이)로 만들어져 있어 자연스러운 빛과 그림자를 통해 실내가 밝아집니다.assistant
삼일절은 대한민국에서 1919년 3월 1일에 일어난 3.1 운동을 기념하는 날입니다. 3.1 운동은 일제 강점기 하에서 한국인들이 독립을 선언하고, 이를 알리기 위해 전국적으로 벌인 대규모 비폭력 시위입니다. 이 운동은 한국 독립운동의 중요한 전환점이 되었으며, 이후 한국의 독립운동에 큰 영향을 미쳤습니다. 삼일절은 한국인들이 자주와 독립을 위해 싸운 역사적 사건을 기리며, 이를 통

#  rag

In [ ]:
# Image 1
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_qdrant import QdrantVectorStore, RetrievalMode
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from uuid import uuid4
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.documents import Document

loader = PyPDFLoader(
    file_path="KISTEP_플라워기술정책대동.pdf",
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

embeddings = OllamaEmbeddings(model="bge-m3")

client = QdrantClient(path="/tmp/0429/test_5")

# 기존 벡터DB 컬렉션 생성
client.create_collection(
    collection_name="kistep_0429_5",
    vectors_config=VectorParams(size=1024, distance=Distance.COSINE),
)

# Image 2
qdrant = QdrantVectorStore(
    client=client,
    collection_name="kistep_0429_5",
    embedding=embeddings,
    retrieval_mode=RetrievalMode.DENSE,
)

uuids = [str(uuid4()) for _ in range(len(splits))]

qdrant.add_documents(documents=splits, ids=uuids)

# 벡터 저장소를 검색기로 설정
retriever = qdrant.as_retriever()

class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

# Image 3
# Define application steps
def retrieve(state: State):
    retrieved_docs = retriever.invoke(state["question"])
    return {"context": retrieved_docs}

def clova_generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    prompt_text = f"""
당신은 Q&A 전문 AI 어시스턴트입니다. 주어진 컨텍스트를 사용하여 질문에 답변해주세요.

컨텍스트:
{docs_content}

질문:
{state["question"]}

답변:
"""
    response = ask_clova_stream(prompt_text)
    return {"answer": response}

In [ ]:
# Compile application and test
clova_graph_builder = StateGraph(State).add_sequence([retrieve, clova_generate])
clova_graph_builder.add_edge(START, "retrieve")
clova_graph = clova_graph_builder.compile()


from IPython.display import Image, display

display(Image(clova_graph.get_graph().draw_mermaid_png()))


In [ ]:
for message, metadata in clova_graph.stream(
    {"question": "트럼프 정부 정책 변화에 따른 국가전략기술 대응 정책을 설명해주세요"}, stream_mode="messages"
):
    print(message.content, end="")
# 3.3s